<a href="https://colab.research.google.com/github/kuromasadev/Home_Sales/blob/main/home_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install findspark
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=8f2dbf64247f5e51db0c9b3fe357be35b519a359e6c389b9f7d3813a0fc2c6b5
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [4]:
import findspark
findspark.init()

In [5]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [6]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
file_path = SparkFiles.get("home_sales_revised.csv")



In [7]:
# 2. Create a temporary view of the DataFrame.

df = spark.read.csv(file_path, header=True, inferSchema=True)

# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

In [8]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
average_prices = spark.sql("""
    SELECT
        YEAR(date) AS year,
        AVG(price) AS avg_price
    FROM
        home_sales
    WHERE
        bedrooms = 4
    GROUP BY
        year
    ORDER BY
        year
""")

average_prices.show()


+----+------------------+
|year|         avg_price|
+----+------------------+
|2019| 300263.6955128205|
|2020|298353.78003169573|
|2021|   301819.44398864|
|2022| 296363.8845050215|
+----+------------------+



In [9]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
average_prices2 = spark.sql("""
    SELECT
        date_built AS year_built,
        AVG(price) AS avg_price
    FROM
        home_sales
    WHERE
        bedrooms = 3 AND bathrooms = 3
    GROUP BY
        year_built
    ORDER BY
        year_built
""")

average_prices2.show(truncate=False)


+----------+------------------+
|year_built|avg_price         |
+----------+------------------+
|2010      |292859.615942029  |
|2011      |291117.46706586826|
|2012      |293683.1872074883 |
|2013      |295962.27145085804|
|2014      |290852.2661870504 |
|2015      |288770.2966101695 |
|2016      |290555.073964497  |
|2017      |292676.7887740029 |
+----------+------------------+



In [10]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

average_prices3 = spark.sql("""
    SELECT
        date_built AS year_built,
        AVG(price) AS avg_price
    FROM
        home_sales
    WHERE
        bedrooms = 3 AND
        bathrooms = 3 AND
        floors = 2 AND
        sqft_living >= 2000
    GROUP BY
        year_built
    ORDER BY
        year_built
""")

average_prices3.show(truncate=False)

+----------+------------------+
|year_built|avg_price         |
+----------+------------------+
|2010      |285010.2215909091 |
|2011      |276553.8128654971 |
|2012      |307539.97402597405|
|2013      |303676.79375      |
|2014      |298264.7183908046 |
|2015      |297609.9679144385 |
|2016      |293965.1046511628 |
|2017      |280317.57692307694|
+----------+------------------+



In [11]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

average_view_rating = spark.sql("""
    SELECT
        AVG(view) AS avg_view_rating
    FROM
        home_sales
    WHERE
        price >= 350000
""").collect()[0]["avg_view_rating"]

end_time = time.time()

print(f"Average View Rating: {average_view_rating:.2f}")
print("--- %s seconds ---" % (end_time - start_time))

Average View Rating: 32.26
--- 0.7893130779266357 seconds ---


In [12]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [13]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [14]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

cached_results = spark.sql("""
    SELECT
        AVG(view) AS avg_view_rating
    FROM
        home_sales
    WHERE
        price >= 350000
""").collect()[0]["avg_view_rating"]

end_time = time.time()

cached_runtime = end_time - start_time


In [15]:
start_time = time.time()

uncached_results = spark.sql("""
    SELECT
        AVG(view) AS avg_view_rating
    FROM
        home_sales
    WHERE
        price >= 350000
""").collect()[0]["avg_view_rating"]

end_time = time.time()

uncached_runtime = end_time - start_time

In [16]:
print(f"Cached Runtime: {cached_runtime:.4f} seconds")
print(f"Uncached Runtime: {uncached_runtime:.4f} seconds")
print(f"Cached Results: {cached_results:.2f}")
print(f"Uncached Results: {uncached_results:.2f}")

Cached Runtime: 0.7879 seconds
Uncached Runtime: 0.4425 seconds
Cached Results: 32.26
Uncached Results: 32.26


In [17]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

output_path = "partitioned_data/"
df.write.partitionBy("date_built").parquet(output_path)

In [18]:
# 11. Read the formatted Parquet data
parquet_path = output_path
parquet_df = spark.read.parquet(parquet_path)

In [19]:
parquet_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- date_built: integer (nullable = true)



In [20]:
# 12. Create a temporary table for the Parquet data
parquet_df.createOrReplaceTempView("parquet_data")

In [21]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

parquet_results = parquet_df.filter(parquet_df["price"] >= 350000) \
                           .groupBy().avg("view") \
                           .collect()[0][0]

end_time = time.time()

parquet_runtime = end_time - start_time

print(f"Parquet Query Runtime: {parquet_runtime:.4f} seconds")
print(f"Parquet Results: {parquet_results:.2f}")

Parquet Query Runtime: 1.0986 seconds
Parquet Results: 32.26


In [22]:
pcached_start_time = time.time()

pcached_results = spark.sql("""
    SELECT
        AVG(view) AS avg_view_rating
    FROM
        home_sales
    WHERE
        price >= 350000
""").collect()[0]["avg_view_rating"]

pcached_end_time = time.time()

pcached_runtime = pcached_end_time - pcached_start_time

print(f"Cached Query Runtime: {pcached_runtime:.4f} seconds")
print(f"Cached Results: {pcached_results:.2f}")

Cached Query Runtime: 0.3760 seconds
Cached Results: 32.26


In [23]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")


In [24]:
# 15. Check if the home_sales is no longer cached

is_cached = spark.catalog.isCached("home_sales")

if is_cached:
    print("The 'home_sales' temporary table is still cached.")
else:
    print("The 'home_sales' temporary table is not cached.")

The 'home_sales' temporary table is not cached.
